In [1]:
import pandas as pd
import urllib.parse
import numpy as np
import re
import os
import csv
#base64解码
import base64
def safe_base64_decode(s):
    for i in range(len(s) % 4):
        s += b'='
    return base64.b64decode(s.encode('utf_8'))
##判断请求方法是否为post，是则尝试进行base64解密，无法解密说明没有数据，或者数据出错，保留原始列数据。
def get_process_post_data(logs):   
    for index,df in logs.iterrows():
        if df['method']=='post':
            try:
                temp=safe_base64_decode(df['post_data'])
                temp=str(temp)
                df['post_data']=urllib.parse.unquote(temp,'utf-8')
            except:
                df['post_data']=df['post_data']
    return logs
#将规则的威胁等级数值化，与owasp一致
def get_threat_level(a):
    if a.find('NOTICE')>-1:
        return 0
    elif a.find('WARNING')>-1:
        return 1
    elif a.find('CRITICAL')>-1:
        return 2
    else:
        return 0

def process_log():
    root = 'C:\\bigData\\datacon\\attack_analysis\\logVicki1'
    file_names = os.listdir(root)
    file_ob_list = []
    for file_name in file_names:
        file_ob = root + '\\' + file_name
        file_ob_list.append(file_ob)
    
    for file_ob in file_ob_list:
        print(file_ob)
        log_clumns=['ip','date','host','method','post_data','parameter','url_path','user_agent','cookie','x-forwarded-for',
            'host_country','host_province','host_city','ip_country','ip_province','ip_city']
        logs = pd.read_csv(file_ob,delimiter=',',names=log_clumns,encoding='utf-8')
        logs = get_process_post_data(logs)
        #创建两个新列
        threat_level=np.repeat(0,logs.shape[0])
        msg=np.repeat('MSG',logs.shape[0])
        logs['threat_level']=threat_level
        logs['msg']=msg
        attack_rule_cloumns=['check_type','rule','msg','severity']
        rules = pd.read_csv('C:\\bigData\\import_file\\OneDrive\\python_file\\attack_rule.csv',delimiter=',',names=attack_rule_cloumns)
        rules['threat_level']=rules.apply(lambda x: get_threat_level(x.severity), axis = 1)
        #对每条记录做正则匹配，此处暂时只检查了post_data，程序成功后，应增加对parameter，cookie，url_path的检查
        for index,rule in rules.iterrows():
            test=0
            if rule['threat_level']==1:
                continue
            try:
                prog = re.compile(rule['rule'],re.MULTILINE)
                #print(prog)
            except:
                continue
            for index,log in logs.iterrows():
                flag = 0
                #print(log['post_data'])
                if rule['check_type'].find('ARGS')>-1 or rule['check_type'].find('REQUEST_BODY')>-1:
                    temp=prog.search(log['post_data'])
                    if temp:
                        flag=flag+1
                        #print(temp.group())
                    temp_param=prog.search(log['parameter'])
                    if temp_param:
                        flag=flag+1
                        #print(temp_param.group())
                if rule['check_type'].find('COOKIE')>-1:
                    temp_cookie = prog.search(log['cookie'])
                    if temp_cookie:
                        flag=flag+1
                        #print(temp_cookie.group())
                if rule['check_type'].find('REQUEST_URI')>-1:
                    temp_path = prog.search(log['url_path'])
                    if temp_path:
                        flag=flag+1
                        #print(temp_path.group())
                if flag>0:
                    logs.at[index, 'threat_level']=log['threat_level']+rule['threat_level']
                    logs.at[index, 'msg']=log['msg']+rule['msg']
                    test=test+1
            if test>10000:
                print(prog)
        print('one log end')
        logs.to_csv(file_ob+'re',index=0)

In [2]:
process_log()

C:\bigData\datacon\attack_analysis\logVicki1\2018-12-12.csv


C:\Users\Simpl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


re.compile('(?i)(?:\\x5c|(?:%(?:c(?:0%(?:[2aq]f|5c|9v)|1%(?:[19p]c|8s|af))|2(?:5(?:c(?:0%25af|1%259c)|2f|5c)|%46|f)|(?:(?:f(?:8%8)?0%8|e)0%80%a|bg%q)f|%3(?:2(?:%(?:%6|4)6|F)|5%%63)|u(?:221[56]|002f|EFC8|F025)|1u, re.IGNORECASE|re.MULTILINE)
re.compile('(?:\\$(?:\\((?:\\(.*\\)|.*)\\)|\\{.*\\})|[<>]\\(.*\\))', re.MULTILINE)
re.compile('(?i)\\b(?:s(?:e(?:t(?:_(?:e(?:xception|rror)_handler|magic_quotes_runtime|include_path)|defaultstub)|ssion_s(?:et_save_handler|tart))|qlite_(?:(?:(?:unbuffered|single|array)_)?query|create_(?:aggrega, re.IGNORECASE|re.MULTILINE)
re.compile('(?:HTTP_(?:ACCEPT(?:_(?:ENCODING|LANGUAGE|CHARSET))?|(?:X_FORWARDED_FO|REFERE)R|(?:USER_AGEN|HOS)T|CONNECTION|KEEP_ALIVE)|PATH_(?:TRANSLATED|INFO)|ORIG_PATH_INFO|QUERY_STRING|REQUEST_URI|AUTH_TYPE)', re.MULTILINE)
re.compile('(?i)\\b(?:i(?:s(?:_(?:in(?:t(?:eger)?|finite)|n(?:u(?:meric|ll)|an)|(?:calla|dou)ble|s(?:calar|tring)|f(?:inite|loat)|re(?:source|al)|l(?:ink|ong)|a(?:rray)?|object|bool)|set)|n(?:(?:clud|vok)e|t(?

re.compile("(?:/\\*!?|\\*/|[';]--|--[\\s\\r\\n\\v\\f]|--[^-]*?-|[^&-]#.*?[\\s\\r\\n\\v\\f]|;?\\\\x00)", re.MULTILINE)
one log end
C:\bigData\datacon\attack_analysis\logVicki1\2018-12-17.csv
re.compile('(?i)(?:\\x5c|(?:%(?:c(?:0%(?:[2aq]f|5c|9v)|1%(?:[19p]c|8s|af))|2(?:5(?:c(?:0%25af|1%259c)|2f|5c)|%46|f)|(?:(?:f(?:8%8)?0%8|e)0%80%a|bg%q)f|%3(?:2(?:%(?:%6|4)6|F)|5%%63)|u(?:221[56]|002f|EFC8|F025)|1u, re.IGNORECASE|re.MULTILINE)
re.compile('(?i)\\b(?:s(?:e(?:t(?:_(?:e(?:xception|rror)_handler|magic_quotes_runtime|include_path)|defaultstub)|ssion_s(?:et_save_handler|tart))|qlite_(?:(?:(?:unbuffered|single|array)_)?query|create_(?:aggrega, re.IGNORECASE|re.MULTILINE)
re.compile('(?:HTTP_(?:ACCEPT(?:_(?:ENCODING|LANGUAGE|CHARSET))?|(?:X_FORWARDED_FO|REFERE)R|(?:USER_AGEN|HOS)T|CONNECTION|KEEP_ALIVE)|PATH_(?:TRANSLATED|INFO)|ORIG_PATH_INFO|QUERY_STRING|REQUEST_URI|AUTH_TYPE)', re.MULTILINE)
re.compile('(?i)\\b(?:i(?:s(?:_(?:in(?:t(?:eger)?|finite)|n(?:u(?:meric|ll)|an)|(?:calla|dou)ble|s(?:c